In [ ]:
pip install functions-framework

In [ ]:
import functions_framework
from pytz import timezone
from datetime import datetime
import pandas as pd
import requests
import sqlalchemy
import pymysql
import os

In [ ]:
@functions_framework.http
def weather_pushing(request):
    schema = "city_scooter_analysis"  # The name of your database
    host = "insert_cloud_SQL_IP"
    user = "root"
    password = "Insert_your_SQL_root_password" # Replace with your actual variable or hardcode for testing
    port = 3306

    # Create the connection string for SQLAlchemy
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # Read the cities table into a DataFrame
    input_city_df = pd.read_sql("city", con=connection_string)
    # existing_weather_df = pd.read_sql("weather", con=connection_string)

    #berlin_timezone = timezone('Europe/Berlin')
    OPENWEATHER_API_KEY = "Insert_your_API_key"  # Replace with your actual API key. Using the string without imports helps debug. In Cloud use Google secrets
    #weather_items = []

    # Setting up lists for later dataframe creation
    city_id = []
    forecast_time = []
    outlook = []
    temperature = []
    temperature_felt = []
    wind_speed = []
    rain_expected_mm = []
    retrieval_time = []

    # Prepping time and source related references
    berlin_timezone = timezone('Europe/Berlin')
    
    url = "https://api.openweathermap.org/data/2.5/forecast"

    for id in input_city_df['city_id']:
        querystring = {"lat": input_city_df.loc[input_city_df['city_id'] == id, :]['latitude'], "lon": input_city_df.loc[input_city_df['city_id'] == id, :]['longitude'], "appid": OPENWEATHER_API_KEY, "units": "metric"}
        retrieval_timestamp = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
        # Reference the sections in the request.
        weather = requests.request("GET", url, params=querystring)
        weather_json = weather.json()

        for element in weather_json['list']:
            city_id.append(id)
            forecast_time.append(element['dt_txt'])
            outlook.append(element['weather'][0]['description'])
            temperature.append(element['main']['temp'])
            temperature_felt.append(element['main']['feels_like'])
            wind_speed.append(element['wind']['speed'])
            try:
                rain_expected_mm.append(element['rain']['3h'])
            except:
                rain_expected_mm.append(0)
            retrieval_time.append(retrieval_timestamp)

    weather_df = pd.DataFrame({"city_id": city_id,
                                        "forecast_time": forecast_time,
                                        "outlook": outlook,
                                        "temperature": temperature,
                                        "temperature_felt": temperature_felt,
                                        "wind_speed": wind_speed,
                                        "rain_expected_mm": rain_expected_mm,
                                        "retrieval_time": retrieval_time}
                                    )
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
    weather_df["retrieval_time"] = pd.to_datetime(weather_df["retrieval_time"]) 

    weather_df.to_sql('weather', if_exists="append", con=connection_string, index=False)

    return "Data has been sent to the table"

In [ ]:
from flask import Request
import json

request_data = {}
request = Request.from_values(data=json.dumps(request_data))

# Call the function
response = weather_pushing(request)
print(response)